### Essential Libraries

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from dataloader import ATeX  # Ensure dataloader.py is in the same directory or in PYTHONPATH
from sklearn.metrics import classification_report
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR


### Dataset Preparation with Augmentation

In [2]:

# Define transformations with augmentation
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean_std[0], mean_std[1]),
])

# Define datasets
train_dataset = ATeX(split="train", transform=data_transforms)
test_dataset = ATeX(split="test", transform=data_transforms)
val_dataset = ATeX(split="val", transform=data_transforms)

# Define DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


### Updated Custom CNN Model

In [3]:

# Updated Custom CNN Model
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn5 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 15)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = self.flatten(x)
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN().to(device)


### Optimizer, Loss, and Scheduler

In [4]:
# Define optimizer, loss function, and learning rate scheduler
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)  # Changed to SGD with momentum
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)


### Training, Validation and Their Evaluation

In [5]:

# Training loop with learning rate scheduler
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels, _ in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    all_val_labels, all_val_preds = [], []
    with torch.no_grad():
        for images, labels, _ in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(preds.cpu().numpy())
    val_loss /= len(val_loader)
    val_report = classification_report(
        all_val_labels, all_val_preds, output_dict=True, zero_division=0)
    val_precision = val_report['weighted avg']['precision']
    val_recall = val_report['weighted avg']['recall']
    val_f1 = val_report['weighted avg']['f1-score']
    
    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, "
          f"Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}")
    
    scheduler.step()


Epoch [1/30] Train Loss: 1.8024, Val Loss: 1.7613, Precision: 0.4859, Recall: 0.4337, F1-Score: 0.4095
Epoch [2/30] Train Loss: 1.4418, Val Loss: 1.6226, Precision: 0.5496, Recall: 0.4952, F1-Score: 0.4830
Epoch [3/30] Train Loss: 1.2997, Val Loss: 1.3076, Precision: 0.6018, Recall: 0.5607, F1-Score: 0.5619
Epoch [4/30] Train Loss: 1.1745, Val Loss: 1.1230, Precision: 0.6589, Recall: 0.6190, F1-Score: 0.6217
Epoch [5/30] Train Loss: 1.0956, Val Loss: 1.1122, Precision: 0.6181, Recall: 0.6142, F1-Score: 0.6029
Epoch [6/30] Train Loss: 0.9242, Val Loss: 0.8861, Precision: 0.7045, Recall: 0.7005, F1-Score: 0.6954
Epoch [7/30] Train Loss: 0.8880, Val Loss: 0.9229, Precision: 0.7008, Recall: 0.6845, F1-Score: 0.6828
Epoch [8/30] Train Loss: 0.8270, Val Loss: 0.8420, Precision: 0.7085, Recall: 0.6933, F1-Score: 0.6911
Epoch [9/30] Train Loss: 0.8135, Val Loss: 0.8715, Precision: 0.7141, Recall: 0.7061, F1-Score: 0.7021
Epoch [10/30] Train Loss: 0.7784, Val Loss: 0.8698, Precision: 0.7105, Re

### Testing and Evaluation

In [6]:

# Evaluate the model on the test set
model.eval()
all_labels, all_preds = [], []
with torch.no_grad():
    for images, labels, _ in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
report = classification_report(all_labels, all_preds, target_names=train_dataset.classes)
print(report)


              precision    recall  f1-score   support

       delta       0.85      0.93      0.89       330
     estuary       0.87      0.83      0.85       125
       flood       0.79      0.86      0.82       283
    glaciers       0.93      0.93      0.93       240
  hot_spring       0.75      0.84      0.79       201
        lake       0.82      0.75      0.78       153
        pool       0.89      0.84      0.86        79
      puddle       0.74      0.61      0.67       168
      rapids       0.86      0.81      0.83       219
       river       0.74      0.63      0.68        73
         sea       0.80      0.90      0.85       108
        snow       0.85      0.82      0.84       142
       swamp       0.90      0.91      0.90       188
   waterfall       0.76      0.73      0.74        96
     wetland       0.98      0.91      0.94        93

    accuracy                           0.84      2498
   macro avg       0.84      0.82      0.83      2498
weighted avg       0.84   

In [7]:
# Save classification report
with open("classification_report_my_modelwithSGD.txt", "w") as f:
    f.write(report)

# Save model weights
torch.save(model.state_dict(), "my_model_cpu.pth")